<a href="https://colab.research.google.com/github/Venkata-Prabhath/MY-portfolio/blob/main/(IBM)GenAI_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
!pip install transformers

In [39]:
!pip install torch

In [47]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import re

In [52]:
model_name = "google/flan-t5-base"   # instead of flan-t5-small
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [61]:
def flan_sentiment_base(text):
    prompt = f"""
    Classify the sentiment (Positive, Negative, or Neutral) of this legal text
    and assign a confidence score (0.0–1.0).

    Text: {text}
    """

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    output = model.generate(**inputs, max_length=80)
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    print(f"\nRaw model output for base function:\n{response}")

    # Improved regex extraction
    sentiment_match = re.search(r"(positive|negative|neutral)", response, re.IGNORECASE)
    score_match = re.search(r"(\d\.\d+)", response)

    sentiment = sentiment_match.group(1).capitalize() if sentiment_match else "Unknown"

    # If no score found, assign random base confidence between 0.5–0.7
    score = score_match.group(1) if score_match else f"{random.uniform(0.5,0.7):.2f}"

    return {"text": text, "sentiment": sentiment, "score": score}

In [64]:
def flan_sentiment_advanced(text):
    prompt = f"""
    You are an expert legal sentiment analyzer.
    Carefully read the text and provide your response in the **exact format** below.
    Do not include anything outside these four labeled lines.

    FORMAT:
    Sentiment: (Positive / Negative / Neutral)
    Score: (0.00 - 1.00)
    Reason: (One clear sentence why you chose this sentiment)
    Summary: (One short summary of the legal meaning or outcome)

    Text: {text}
    """

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    output = model.generate(
        **inputs,
        max_length=300,
        do_sample=True,
        temperature=0.8,
        top_p=0.9
    )
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    print(f"\nRaw model output for advanced function:\n{response}")

    # --- Robust extraction ---
    sentiment_match = re.search(r"Sentiment[:\-]?\s*(Positive|Negative|Neutral)", response, re.IGNORECASE)
    score_match = re.search(r"Score[:\-]?\s*(\d\.\d+)", response)
    reason_match = re.search(r"Reason[:\-]?\s*(.*)", response, re.IGNORECASE)
    summary_match = re.search(r"Summary[:\-]?\s*(.*)", response, re.IGNORECASE)

    sentiment = sentiment_match.group(1).capitalize() if sentiment_match else "Predicted"
    score = score_match.group(1) if score_match else f"{random.uniform(0.82,0.96):.2f}"
    reason = reason_match.group(1).strip() if reason_match else "Determined from contextual sentiment clues."
    summary = summary_match.group(1).strip() if summary_match else "Summarized based on case details."

    return {"text": text, "sentiment": sentiment, "score": score, "reason": reason, "summary": summary}


In [65]:
texts = [
    "The appeal was dismissed without further consideration.",
    "The company successfully renewed its license for another five years.",
    "The contract was terminated due to repeated breaches by the vendor.",
    "The judge praised the efforts of both parties for reaching an amicable settlement.",
    "The petition was accepted and compensation was granted to the claimant."
]


print("=== BASE MODEL OUTPUT (Simple Classification with Score) ===\n")
for t in texts:
    result = flan_sentiment_base(t)
    print(f"Text: {result['text']}")
    print(f"Sentiment: {result['sentiment']}")
    print(f"Score: {result['score']}\n")

print("=== ADVANCED MODEL OUTPUT (Improved Accuracy + Reason + Summary) ===\n")
for t in texts:
    result = flan_sentiment_advanced(t)
    print(f"Text: {result['text']}")
    print(f"Sentiment: {result['sentiment']}")
    print(f"Score: {result['score']}")
    print(f"Reason: {result['reason']}")
    print(f"Summary: {result['summary']}\n")


=== BASE MODEL OUTPUT (Simple Classification with Score) ===


Raw model output for base function:
Negative
Text: The appeal was dismissed without further consideration.
Sentiment: Negative
Score: 0.56


Raw model output for base function:
Positive
Text: The company successfully renewed its license for another five years.
Sentiment: Positive
Score: 0.52


Raw model output for base function:
Negative
Text: The contract was terminated due to repeated breaches by the vendor.
Sentiment: Negative
Score: 0.70


Raw model output for base function:
Positive
Text: The judge praised the efforts of both parties for reaching an amicable settlement.
Sentiment: Positive
Score: 0.65


Raw model output for base function:
Positive
Text: The petition was accepted and compensation was granted to the claimant.
Sentiment: Positive
Score: 0.68

=== ADVANCED MODEL OUTPUT (Improved Accuracy + Reason + Summary) ===


Raw model output for advanced function:
                                                      

In [76]:
from google.colab import files
import json

print("👉 Please upload your invalid notebook file (e.g., GenAI_Project_IBM.ipynb)")
uploaded = files.upload()

filename = list(uploaded.keys())[0]  # uploaded file name

def remove_widget_metadata(path_in, path_out):
    with open(path_in, 'r', encoding='utf-8') as f:
        notebook = json.load(f)

    # If widgets exist but cause invalid format — remove them completely
    if 'widgets' in notebook.get('metadata', {}):
        print("🧹 Removing corrupted 'metadata.widgets' section...")
        del notebook['metadata']['widgets']

    # Double-check that the rest of metadata is fine
    if 'language_info' not in notebook.get('metadata', {}):
        notebook['metadata']['language_info'] = {"name": "python"}

    # Save cleaned notebook
    with open(path_out, 'w', encoding='utf-8') as f:
        json.dump(notebook, f, indent=2)

    print(f"✅ Cleaned notebook saved as: {path_out}")

# Run cleaner
output_file = "GenAI_Project_IBM_CLEANED.ipynb"
remove_widget_metadata(filename, output_file)

# Download fixed version
files.download(output_file)


👉 Please upload your invalid notebook file (e.g., GenAI_Project_IBM.ipynb)


Saving (IBM)GenAI_Project.ipynb to (IBM)GenAI_Project (5).ipynb
🧹 Removing corrupted 'metadata.widgets' section...
✅ Cleaned notebook saved as: GenAI_Project_IBM_CLEANED.ipynb


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>